In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import json
from keras.preprocessing.text import Tokenizer
import numpy as np 
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import precision_recall_fscore_support, top_k_accuracy_score


In [2]:
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/avani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
with open("training_data.jsonl") as f:
    training_data = json.load(f)
with open("test_data.jsonl") as f:
    testing_data = json.load(f)
with open("validation_data.jsonl") as f:
    valid_data = json.load(f)

In [4]:
train_data = []

for i in range(len(training_data)):
    train_data.append(training_data[i][0][0] + ' ' + training_data[i][0][2])
    
print(len(train_data))

1386


In [5]:
val_data = []

for i in range(len(valid_data)):
    val_data.append(valid_data[i][0][0] + ' ' + valid_data[i][0][2])
    
print(len(val_data))

175


In [6]:
test_data = []

for i in range(len(testing_data)):
    test_data.append(testing_data[i][0][0] + ' ' + testing_data[i][0][2])

print(len(test_data))

183


In [7]:
REPLACE_BY_SPACE = re.compile('[/(){}\[\]\|@,;.-_-—]')

def clean_text(text):
    
    text = text.lower()
    text = REPLACE_BY_SPACE.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

train_data = list(map(clean_text, train_data))
val_data = list(map(clean_text, val_data))
test_data = list(map(clean_text, test_data))

print(len(train_data))
print(len(val_data))
print(len(test_data))

1386
175
183


In [8]:
diction = {}
for i in training_data:
    diction[i[1]] = 1
    
print(len(diction))

j = 1
for i in diction:
    diction[i] = j
    j += 1

print(diction)

20
{'generation': 1, 'sentiment analysis, stylistic analysis, and argument mining': 2, 'information extraction, retrieval and text mining': 3, 'resources and evaluation': 4, 'machine learning for nlp': 5, 'interpretability and analysis of models for nlp': 6, 'question answering': 7, 'linguistic theories, cognitive modeling and psycholinguistics': 8, 'discourse and pragmatics': 9, 'machine translation and multilinguality': 10, 'ethics in nlp': 11, 'summarization': 12, 'tagging, chunking, syntax and parsing': 13, 'dialogue and interactive systems': 14, 'semantics': 15, 'computational social science, social media and cultural analytics': 16, 'nlp applications': 17, 'language grounding to vision, robotics and beyond': 18, 'phonology, morphology and word segmentation': 19, 'speech and multimodality': 20}


In [9]:
train_labels = []
for i in range(len(training_data)):
    train_labels.append(diction[training_data[i][1]])
    
print(len(train_labels))

val_labels = []
for i in range(len(valid_data)):
    val_labels.append(diction[valid_data[i][1]])
    
print(len(val_labels))

test_labels = []
for i in range(len(testing_data)):
    test_labels.append(diction[testing_data[i][1]])

print(len(test_labels))

1386
175
183


In [10]:
y_train = tf.convert_to_tensor(train_labels, dtype = tf.int64)
y_val = tf.convert_to_tensor(val_labels, dtype = tf.int64)
y_test = tf.convert_to_tensor(test_labels, dtype = tf.int64)

In [11]:
import gensim
from gensim.models import Word2Vec

w2vmodel = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

print(w2vmodel)
# summarize vocabulary size in model
# words = list(embed.wv)
# print('Vocabulary size: %d' % len(words))


In [12]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

t_data = []
for idx in range(len(train_data)):
  x = nltk.tokenize.word_tokenize(train_data[idx])
  temp = []
  for token in x:
    # w2vmodel[i]
    if(token in w2vmodel):
      temp.append(w2vmodel[token])
  t_data.append(temp)


v_data = []
for idx in range(len(val_data)):
  x = nltk.tokenize.word_tokenize(val_data[idx])
  temp = []
  for token in x:
    # w2vmodel[i]
    if(token in w2vmodel):
      temp.append(w2vmodel[token])
  v_data.append(temp)

ts_data = []
for idx in range(len(test_data)):
  x = nltk.tokenize.word_tokenize(test_data[idx])
  temp = []
  for token in x:
    # w2vmodel[i]
    if(token in w2vmodel):
      temp.append(w2vmodel[token])
  ts_data.append(temp)


[nltk_data] Downloading package punkt to /home/avani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(t_data,dtype='float64')
print(len(X_train[1]))
X_val =tf.keras.preprocessing.sequence.pad_sequences(v_data, maxlen = X_train.shape[1],dtype='float64')
print(len(X_val[1]))
X_test =tf.keras.preprocessing.sequence.pad_sequences(ts_data, maxlen = X_train.shape[1],dtype='float64')
print(len(X_test[1]))

179
179
179


In [16]:
model_w2v = Sequential()
# model_w2v.add(Embedding(len(tokenizer.word_index) + 1, 300, input_length = t_data_pad.shape[1]))
model_w2v.add(Input((179, 300 )))

model_w2v.add(SpatialDropout1D(0.2))
model_w2v.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model_w2v.add(Dense(21, activation = 'softmax'))
model_w2v.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model_w2v.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spatial_dropout1d_1 (Spatia  (None, 179, 300)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 21)                2121      
                                                                 
Total params: 162,521
Trainable params: 162,521
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
epochs = 100
batch_size = 32
checkpoint_path = 'best_w2v_model.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history_w2v = model_w2v.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = (X_val, y_val),callbacks=[model_checkpoint_callback])

Epoch 1/100
44/44 [==============================] - 28s 347ms/step - loss: 2.8742 - accuracy: 0.1032 - val_loss: 2.7766 - val_accuracy: 0.1200
Epoch 2/100
44/44 [==============================] - 10s 223ms/step - loss: 2.6929 - accuracy: 0.1811 - val_loss: 2.7599 - val_accuracy: 0.1543
Epoch 3/100
44/44 [==============================] - 9s 195ms/step - loss: 2.6178 - accuracy: 0.2114 - val_loss: 2.5200 - val_accuracy: 0.2000
Epoch 4/100
44/44 [==============================] - 9s 213ms/step - loss: 2.6165 - accuracy: 0.2143 - val_loss: 2.5080 - val_accuracy: 0.2457
Epoch 5/100
44/44 [==============================] - 8s 189ms/step - loss: 2.4083 - accuracy: 0.2893 - val_loss: 2.4392 - val_accuracy: 0.2400
Epoch 6/100
44/44 [==============================] - 8s 184ms/step - loss: 2.3208 - accuracy: 0.3088 - val_loss: 2.5677 - val_accuracy: 0.2000
Epoch 7/100
44/44 [==============================] - 8s 184ms/step - loss: 2.3041 - accuracy: 0.3182 - val_loss: 2.2906 - val_accuracy: 0.31

Epoch 58/100
44/44 [==============================] - 9s 202ms/step - loss: 0.1717 - accuracy: 0.9582 - val_loss: 2.1632 - val_accuracy: 0.4857
Epoch 59/100
44/44 [==============================] - 8s 179ms/step - loss: 0.1595 - accuracy: 0.9632 - val_loss: 2.3281 - val_accuracy: 0.4286
Epoch 60/100
44/44 [==============================] - 8s 177ms/step - loss: 0.1765 - accuracy: 0.9603 - val_loss: 2.1095 - val_accuracy: 0.4914
Epoch 61/100
44/44 [==============================] - 8s 181ms/step - loss: 0.1514 - accuracy: 0.9697 - val_loss: 2.2210 - val_accuracy: 0.4514
Epoch 62/100
44/44 [==============================] - 9s 209ms/step - loss: 0.1424 - accuracy: 0.9704 - val_loss: 2.2447 - val_accuracy: 0.4571
Epoch 63/100
44/44 [==============================] - 8s 186ms/step - loss: 0.1378 - accuracy: 0.9683 - val_loss: 2.3678 - val_accuracy: 0.4571
Epoch 64/100
44/44 [==============================] - 8s 187ms/step - loss: 0.1553 - accuracy: 0.9596 - val_loss: 2.1808 - val_accuracy:

In [18]:
model_w2v.load_weights('best_w2v_model.hdf5')

model_w2v.evaluate(X_val,y_val)

6/6 [==============================] - 0s 40ms/step - loss: 2.3656 - accuracy: 0.5086


[2.365574836730957, 0.508571445941925]

In [19]:
model_w2v.evaluate(X_test,y_test)

6/6 [==============================] - 0s 41ms/step - loss: 2.3191 - accuracy: 0.5355


[2.31908917427063, 0.5355191230773926]

In [20]:
y_test_pred = model_w2v.predict(X_test)

In [21]:
print("Top 1 accuracy: ", top_k_accuracy_score( y_true = y_test, y_score = y_test_pred, k=1, labels = np.array(range(0, 21, 1)) ) )
print("Top 3 accuracy: ", top_k_accuracy_score( y_true = y_test, y_score = y_test_pred, k=3, labels = np.array(range(0, 21, 1)) ) )
print("Top 5 accuracy: ", top_k_accuracy_score( y_true = y_test, y_score = y_test_pred, k=5, labels = np.array(range(0, 21, 1)) ) )
    
print("Precision, Recall, F-1:", precision_recall_fscore_support(y_true = y_test, y_pred = np.argmax(y_test_pred, axis = 1), average='weighted'))

Top 1 accuracy:  0.5355191256830601
Top 3 accuracy:  0.7486338797814208
Top 5 accuracy:  0.819672131147541
Precision, Recall, F-1: (0.5262265057347025, 0.5355191256830601, 0.5177966816940802, None)


/home/avani/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
